In [3]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import shap
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large", use_fast=True)
model =  AutoModelForCausalLM.from_pretrained("gpt2").cuda()
def f(x):
    return model(x)

In [9]:
# set model decoder to true
model.config.is_decoder=True
# set text-generation params under task_specific_params
model.config.task_specific_params["text-generation"] = {
    "do_sample": True,
    "max_length": 50,
    "temperature": 0.7,
    "top_k": 50,
    "no_repeat_ngram_size": 2
  }

In [11]:
explainer = shap.KernelExplainer(model,shap.maskers.Text(tokenizer, mask_token='<|endoftext|>'))

In [17]:
s = ['What would I do without you']
shap_exp = explainer(s)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
shap.plots.text(shap_exp)

,What,would,I,do,without,you
?,0.071,1.525,-0.152,2.241,2.654,5.047
,-0.07,-0.064,-0.145,0.254,0.511,-0.122
,-0.001,0.235,-0.328,-0.153,-0.396,-0.063
The,-0.12,-0.178,-0.237,0.011,-0.053,-0.727
man,-0.062,0.019,0.218,0.243,0.165,1.066
was,0.024,0.301,0.111,0.047,0.252,-0.511
a,-0.104,-0.037,0.194,-0.094,-0.045,0.23
tall,0.033,0.149,0.038,-0.018,-0.138,-0.123
",",-0.171,-0.065,-0.164,-0.06,-0.006,-0.03
bald,0.043,-0.038,0.049,-0.035,-0.119,-0.122


In [ ]:
def f(x):
    # print(x.shape)
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128,truncation=True) for v in x]).cuda()
    # tv = torch.tensor(x).cuda(0)
    attention_mask = (tv!=0).type(torch.int64).cuda(0)
    outputs = parallel_model(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [ ]:
# explainer = shap.Explainer(f,tokenizer,output_names=labels)
background_samples = \
    torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128,truncation=True) for v in data['text'][:100]]).numpy()
print(background_samples.shape)


In [ ]:
explainer = shap.Explainer(f, tokenizer, output_names=labels)

In [ ]:
shap_values = explainer(data['text'][100:110].to_numpy())
# to_test = \
#     torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128,truncation=True) for v in data['text'][100:110]]).numpy()
# shap_vals = explainer(to_test)
# explainer.data

In [ ]:
# print(len(shap_vals))
# print(shap_vals[:, :, 0].shape)
# print(shap_vals[:, :, 'joy'].mean(0).shape)
# [x.shape for x in shap_vals]
# ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
tokens = tokenizer.convert_ids_to_tokens(to_test[3])
tokens = tokens[:tokens.index('[PAD]')]
print(tokens)
first_conv_explanation = shap.Explanation(shap_vals[2][3][:len(tokens)], data=tokens)

In [ ]:
# shap.plots.bar(first_conv_explanation)
shap.plots.bar(shap_values[:,:,'surprise'].mean(0))

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_vals[1])